# Before you start:
This notebook shows how to execute YOLOv7 to detect loading bay door positions in video. The data for this project
was annotated using RoboFlow. 

The data can be accessed here: https://app.roboflow.com/james-skelton/ballhandler-basketball/overview

and you can download it with:

> $ curl -L "https://app.roboflow.com/ds/yjvwwIdZlU?key=I3Dz8Y1jHy" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


Miscellaneous notes:
- Go to "Train" to train a model from scratch, "Test" to assess the quality of a trained model, and "Detect" to run detection on a sample. 
- To access the training data & pretrained model for this repo, run the following cell. You will then need to navigate to the "Helpers" section, and follow the instructions there to set up your code for YOLOv7. 

In [ ]:
!curl -L "https://app.roboflow.com/ds/yjvwwIdZlU?key=I3Dz8Y1jHy" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
! pip install gdown
! gdown 164x37XUfwdo0NK8yHWUe69VnKslemvKn
! gdown 1C7IKcnlhfyUIG8d5SoiiY026Z4BCYRCx
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
!pip install -r requirements.txt
!pip install setuptools==59.5.0
!pip install torchvision==0.11.3+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

## Helpers

To set up your RoboFlow annotated data for YOLO, we've provided some helper functions to quickly clean up the filenames. 
Simply run all the cells below

In [ ]:
! mkdir v-test
! mv train/ v-test/
! mv test/ v-test/
! mv valid/ v-test/

In [ ]:
import os 

# remove roboflow extra junk

count = 0
for i in sorted(os.listdir('v-test/train/labels')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = 'v-test/train/labels/'+i
    dest = 'v-test/train/labels/'+j[0]+dict1[count]+'.txt'
    os.rename(source, dest)
    
count = 0
for i in sorted(os.listdir('v-test/train/images')):
    if count >=3:
        count = 0
    count += 1
    if i[0] == '.':
        continue
    j = i.split('_')
    dict1 = {1:'a', 2:'b', 3:'c'}
    source = 'v-test/train/images/'+i
    dest = 'v-test/train/images/'+j[0]+dict1[count]+'.jpg'
    os.rename(source, dest)
    
for i in sorted(os.listdir('v-test/valid/labels')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = 'v-test/valid/labels/'+i
    dest = 'v-test/valid/labels/'+j[0]+'.txt'
    os.rename(source, dest)
    
for i in sorted(os.listdir('v-test/valid/images')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = 'v-test/valid/images/'+i
    dest = 'v-test/valid/images/'+j[0]+'.jpg'
    os.rename(source, dest)
for i in sorted(os.listdir('v-test/test/labels')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = 'v-test/test/labels/'+i
    dest = 'v-test/test/labels/'+j[0]+'.txt'
    os.rename(source, dest)
    
for i in sorted(os.listdir('v-test/test/images')):
    if i[0] == '.':
        continue
    j = i.split('_')
    source = 'v-test/test/images/'+i
    dest = 'v-test/test/images/'+j[0]+'.jpg'
    os.rename(source, dest)

## Train


This section shows how to fine tune a model for a custom dataset.

### Training instructions & tips
- If you need to change the locations of your training/validation/test images, then be sure to go to custom.yaml in the "data" folder, and change the path locations. 
- also in custom.yaml, you can set and label the number of labels you want to be able to detect with your model
- If you are on a distributed machine, use the second train command in the cell below. Be sure to change "nproc_per_node" to accurately reflect the number of GPUs on your device. 
- use the hyp.scratch.custom.yaml file to change hyperparameters for training. 

In [ ]:
!python train.py --workers 8 --device 0 --batch-size 2 --data data/coco.yaml --img 640 360 --cfg cfg/training/yolov7x.yaml --weights yolov7_training.pt --name yolov7-ballhandler --hyp data/hyp.scratch.custom.yaml --epochs 50

# !python -m torch.distributed.launch --nproc_per_node 2 --master_port 9527 train.py --workers 16 --device 0,1 --sync-bn --batch-size 8 --data data/coco.yaml --img 1280 720 --cfg cfg/training/yolov7.yaml --weights yolov7_training.pt --name yolov7-ballhandler --hyp data/hyp.scratch.custom.yaml --epochs 50


## Detect

Use the following cell to run detection on a submitted image. 

- Change image or video being detected on using --source tag
- img size X dimension must be correct for this to run. no Y needed
- if you want to do detection on anything other than the test set, you will need to upload the video to the platform using a standard file upload in the top left corner. 

In [ ]:
#your model
!python detect.py --weights runs/train/yolov7-ballhandler/weights/best.pt --conf 0.25 --img-size 640 --source "testvid-nba.mp4" --name test
#pretrained
!python detect.py --weights best_basketball.pt --conf 0.25 --img-size 640 --source "testvid-nba.mp4" --name test


## Testing

In [ ]:
# your model
!python test.py --data data/test.yaml --img 640 --batch 16 --conf 0.001 --iou 0.65 --device 0 --weights runs/train/yolov7-ballhandler/weights/best.pt --name yolov7_ballhandler_testing

#pretrained
!python test.py --data data/test.yaml --img 640 --batch 16 --conf 0.001 --iou 0.65 --device 0 --weights best_basketball.pt --name yolov7_ballhandler_testing
